
# HW3 Task 1 — K‑Means with Euclidean, Cosine, and Generalized Jaccard

This notebook implements K‑Means **from scratch** with three metrics and answers Q1–Q5.

**Data expected** (place alongside this notebook or update the paths):
- `/mnt/data/data.csv` — shape: 10000 × 784
- `/mnt/data/label.csv` — 10000 labels, 10 classes

We follow the stop criteria from the assignment:
- Stop when **no change in centroid position**, **or** when **SSE increases** at the next iteration, **or** when **max iterations** is reached.

> **Generalized Jaccard** is implemented as \(1 - \frac{\sum_i \min(x_i, y_i)}{\sum_i \max(x_i, y_i)}\) (non‑negative vectors).


In [1]:

import numpy as np
import pandas as pd
import time

EPS = 1e-12

# ---- Load ----
X = pd.read_csv('data.csv', header=None).values.astype(float)
y = pd.read_csv('label.csv', header=None).iloc[:,0].values
K = len(np.unique(y))

def euclidean_sq_dists(A, B):
    A2 = np.sum(A*A, axis=1, keepdims=True)
    B2 = np.sum(B*B, axis=1, keepdims=True).T
    d2 = A2 + B2 - 2 * (A @ B.T)
    np.maximum(d2, 0, out=d2)
    return d2

def cosine_dists(A, B):
    An = np.linalg.norm(A, axis=1, keepdims=True) + EPS
    Bn = np.linalg.norm(B, axis=1, keepdims=True).T + EPS
    sims = (A @ B.T) / (An * Bn)
    sims = np.clip(sims, -1.0, 1.0)
    return 1.0 - sims

def generalized_jaccard_dists(A, B):
    n, d = A.shape
    m = B.shape[0]
    out = np.empty((n, m), dtype=float)
    for j in range(m):
        Bj = B[j]
        mins = np.minimum(A, Bj)
        maxs = np.maximum(A, Bj) + EPS
        out[:, j] = 1.0 - (mins.sum(axis=1) / maxs.sum(axis=1))
    np.maximum(out, 0, out=out)
    return out

def kmeans(X, K, metric="euclidean", max_iter=500, tol=1e-6, init="random", stop_mode="combined", random_state=42):
    rng = np.random.default_rng(random_state)
    if metric == "euclidean":
        dist_fn = lambda A, B: np.sqrt(np.clip(euclidean_sq_dists(A, B), 0, np.inf))
    elif metric == "cosine":
        dist_fn = cosine_dists
    elif metric == "jaccard":
        dist_fn = generalized_jaccard_dists
    else:
        raise ValueError("Unknown metric")
    # init
    idx = rng.choice(X.shape[0], size=K, replace=False)
    C = X[idx].copy()
    if metric == "cosine":
        C /= (np.linalg.norm(C, axis=1, keepdims=True) + EPS)
    sse_history = []
    start = time.time()
    for it in range(1, max_iter + 1):
        D = dist_fn(X, C)
        labels = np.argmin(D, axis=1)
        dmin = D[np.arange(X.shape[0]), labels]
        sse = float(np.sum(dmin**2))
        sse_history.append(sse)
        new_C = np.zeros_like(C)
        for k in range(K):
            pts = X[labels == k]
            new_C[k] = X[rng.integers(0, X.shape[0])] if len(pts) == 0 else pts.mean(axis=0)
        if metric == "cosine":
            new_C /= (np.linalg.norm(new_C, axis=1, keepdims=True) + EPS)
        centroid_shift = float(np.linalg.norm(new_C - C))
        sse_increase = len(sse_history) >= 2 and (sse_history[-1] > sse_history[-2] + 1e-12)
        stop = False
        if stop_mode == "combined":
            stop = (centroid_shift < tol) or sse_increase or (it == max_iter)
        elif stop_mode == "no_centroid_change":
            stop = (centroid_shift < tol)
        elif stop_mode == "sse_increase":
            stop = sse_increase
        elif stop_mode == "max_iter_only":
            stop = (it == max_iter)
        else:
            raise ValueError("Unknown stop_mode")
        C = new_C
        if stop:
            break
    elapsed = time.time() - start
    return {"labels": labels, "centroids": C, "sse_history": sse_history,
            "iters": it, "elapsed": elapsed, "final_sse": sse_history[-1]}

def majority_vote_accuracy(pred_clusters, true_labels, K):
    acc = 0
    mapping = {}
    for k in range(K):
        idx = np.where(pred_clusters == k)[0]
        if len(idx) == 0:
            mapping[k] = None
            continue
        labs, counts = np.unique(true_labels[idx], return_counts=True)
        maj = labs[np.argmax(counts)]
        mapping[k] = int(maj)
        acc += (true_labels[idx] == maj).sum()
    return acc / len(true_labels), mapping
print("Loaded X:", X.shape, "K:", K)

Loaded X: (10000, 784) K: 10


In [2]:

# Q1–Q3 runs (combined stopping rule). You can raise max_iter for more thorough runs.
results = {}
for metric, max_iter in [("euclidean", 50), ("cosine", 50), ("jaccard", 20)]:
    out = kmeans(X, K, metric=metric, max_iter=max_iter, init="random", stop_mode="combined", random_state=1)
    acc, mapping = majority_vote_accuracy(out["labels"], y, K)
    out["accuracy"] = acc
    out["label_map"] = mapping
    results[metric] = out

summary_combined = pd.DataFrame({
    "Metric": ["Euclidean", "Cosine", "Jaccard"],
    "Final_SSE": [results["euclidean"]["final_sse"], results["cosine"]["final_sse"], results["jaccard"]["final_sse"]],
    "Iterations": [results["euclidean"]["iters"], results["cosine"]["iters"], results["jaccard"]["iters"]],
    "Time_sec": [results["euclidean"]["elapsed"], results["cosine"]["elapsed"], results["jaccard"]["elapsed"]],
    "Accuracy": [results["euclidean"]["accuracy"], results["cosine"]["accuracy"], results["jaccard"]["accuracy"]],
})
summary_combined

,Metric,Final_SSE,Iterations,Time_sec,Accuracy
0,Euclidean,2.543218e+10,37,8.024754,0.6036
1,Cosine,6.821488e+02,38,6.057231,0.6118
2,Jaccard,3.955792e+03,2,1.487061,0.5220



### Our run results here (Q1–Q3, combined stop)

| Metric | Final_SSE | Iterations | Time_sec | Accuracy |
|---|---:|---:|---:|---:|
| Euclidean | 2.543e+10 | 37 | 16.70 | 0.6036 |
| Cosine    | 682.1 | 38 | 15.95 | 0.6118 |
| Jaccard   | 3956 | 2 | 4.70 | 0.5220 |

> *Note:* We used random init for speed in this notebook cell; feel free to switch to k‑means++ and larger `max_iter` locally to reproduce/extend.


In [3]:

# Q4: Compare SSEs under each *specific* condition.
# NOTE: Full-data, full-iteration runs can be time-consuming; adjust caps as needed.
def q4_run(metric, max_iter_cap):
    rows = []
    outA = kmeans(X, K, metric=metric, max_iter=max_iter_cap, stop_mode="no_centroid_change", random_state=1)
    rows.append({"Metric": metric.capitalize(), "Stop_Condition": "No centroid change", "Final_SSE": outA["final_sse"], "Iterations": outA["iters"], "Time_sec": outA["elapsed"]})
    outB = kmeans(X, K, metric=metric, max_iter=max_iter_cap, stop_mode="sse_increase", random_state=1)
    rows.append({"Metric": metric.capitalize(), "Stop_Condition": "SSE increases", "Final_SSE": outB["final_sse"], "Iterations": outB["iters"], "Time_sec": outB["elapsed"]})
    outC = kmeans(X, K, metric=metric, max_iter=100, stop_mode="max_iter_only", random_state=1)
    rows.append({"Metric": metric.capitalize(), "Stop_Condition": "Max iters (100)", "Final_SSE": outC["final_sse"], "Iterations": outC["iters"], "Time_sec": outC["elapsed"]})
    return rows

rows = []
rows += q4_run("euclidean", 50)
rows += q4_run("cosine", 50)
rows += q4_run("jaccard", 20)
q4_table = pd.DataFrame(rows)
q4_table

,Metric,Stop_Condition,Final_SSE,Iterations,Time_sec
0,Euclidean,No centroid change,2.543218e+10,37,6.926116
1,Euclidean,SSE increases,2.543218e+10,50,7.488884
2,Euclidean,Max iters (100),2.543218e+10,100,17.203378
3,Cosine,No centroid change,6.822584e+02,50,7.827473
4,Cosine,SSE increases,6.821488e+02,38,6.672509
5,Cosine,Max iters (100),6.820810e+02,100,16.214601
6,Jaccard,No centroid change,3.720043e+03,20,16.199091
7,Jaccard,SSE increases,3.955792e+03,2,2.049289
8,Jaccard,Max iters (100),3.719765e+03,100,81.430011



## Q5 — Observations / Takeaways

- **Euclidean vs Cosine**: On this dataset (high‑dimensional, non‑negative), Cosine distance often yields competitive or slightly higher accuracy than Euclidean because it focuses on **direction** (shape/pattern) rather than magnitude.  
- **Generalized Jaccard**: Works with non‑negative features and is robust to sparsity, but its centroid update with simple means is a heuristic; convergence may be faster/slower depending on initialization, and accuracy is commonly lower than Euclidean/Cosine for grayscale‑like vectors.  
- **SSE comparability**: We report SSE as the **sum of squared metric distances** for each variant; absolute scales differ across metrics, so **compare within‑metric across iterations** and use **accuracy** for cross‑metric prediction quality.  
- **Stopping rules**: 'No centroid change' usually halts earlier than a strict max‑iter rule; 'SSE increases' is a safe early‑stop that prevents divergence with non‑Euclidean metrics.  
- **Initialization**: Better seeding (k‑means++) and multiple restarts materially improve stability and accuracy for all three metrics.  
